# Process all file in folder





In [1]:
%load_ext autoreload
%autoreload 2

List the files to process in both conditions wt and sega

In [2]:
import pandas as pd

from pathlib import Path

# list files from the src folder
src = Path('/media/cephfs2/jparham/Joe for Jerome /')
dst = Path('./')

filelist = pd.concat((
    pd.DataFrame.from_records(
        [{'path':f, 'name':f.name, 'condition':'sega'}  for d in src.glob('*/*/Crop*') for f in d.glob('[!.]*.tif')]
    ),
    pd.DataFrame.from_records(
    [{'path':f, 'name':f.name, 'condition':'wt'}  for d in src.glob('Div*') for f in d.glob('[!.]*.tif')]
)),axis=0,ignore_index=True)

print(f"Number of files {len(filelist)}")
filelist.head()


Number of files 62


,path,name,condition
0,/media/cephfs2/jparham/Joe for Jerome /SegA Li...,01a_delSegA_CM_SyS_reg_Cell_0.tif,sega
1,/media/cephfs2/jparham/Joe for Jerome /SegA Li...,01a_delSegA_CM_SyS_reg_Cell_12.tif,sega
2,/media/cephfs2/jparham/Joe for Jerome /SegA Li...,01a_delSegA_CM_SyS_reg_Cell_13.tif,sega
3,/media/cephfs2/jparham/Joe for Jerome /SegA Li...,01a_delSegA_CM_SyS_reg_Cell_3.tif,sega
4,/media/cephfs2/jparham/Joe for Jerome /SegA Li...,01a_delSegA_CM_SyS_reg_Cell_11.tif,sega


## Process all file

In [ ]:
from tqdm.notebook import tqdm
tqdm.pandas()
import motionquant as mq
import dask

results_path = dst/Path('results.h5') # result
if results_path.exists():
    results_path.unlink()

print(f"Saving results in {results_path}")

df = []
for filename in tqdm(filelist['path'].iloc):
    try:
        name = filename.stem        
        img, mask, position, speed, diff, flow, rho, div, blob = mq.process(filename)
        df.append(mq.record(name, img, mask, position, speed, diff, flow, rho, div, blob))
        mq.save_result(results_path, name, img, mask, position, speed, diff, flow, rho, div, blob)
    except Exception as e:
        print(f"file '{filename}' could not be processed")
        print(e)
        pass


## Parallel processing

In [3]:
from dask.distributed import LocalCluster
cluster = LocalCluster()          # Fully-featured local Dask cluster
client = cluster.get_client()
cluster.scale(8)

In [4]:
import motionquant as mq
import dask
from dask.distributed import Lock
import torch

def process(filename, results_path):
    name = filename.stem        
    img, mask, position, speed, diff, flow, rho, div, blob = mq.process(filename)
    lock = Lock('process-sufo')
    lock.acquire()
    mq.save_result(results_path, name, img, mask, position, speed, diff, flow, rho, div, blob)
    lock.release()
    df = mq.record(name, img, mask, position, speed, diff, flow, rho, div, blob)
    return df

def process_safe(filename, results_path):
    try:
        return process(filename, results_path)
    except Exception as e:
        print(f"file '{filename}' could not be processed")
        print(e)
        pass


results_path = dst/Path('results.h5') # result
if results_path.exists():
    results_path.unlink()

print(f"Saving results in {results_path}")
# process(filelist['path'].iloc[0], results_path)
tsk = [dask.delayed(process_safe)(filename, results_path) for filename in filelist['path'].iloc]
res = dask.compute(tsk)


Saving results in results.h5


file '/media/cephfs2/jparham/Joe for Jerome /SegA Live imaging /20240229_delSegA/CroppedDivisions/01a_delSegA_CM_SyS_reg_Cell_11.tif' could not be processed
HDF5 error back trace

  File "H5F.c", line 836, in H5Fopen
    unable to synchronously open file
  File "H5F.c", line 796, in H5F__open_api_common
    unable to open file
  File "H5VLcallback.c", line 3863, in H5VL_file_open
    open failed
  File "H5VLcallback.c", line 3675, in H5VL__file_open
    open failed
  File "H5VLnative_file.c", line 128, in H5VL__native_file_open
    unable to open file
  File "H5Fint.c", line 1873, in H5F_open
    unable to lock the file
  File "H5FD.c", line 2034, in H5FD_lock
    driver lock request failed
  File "H5FDsec2.c", line 988, in H5FD__sec2_lock
    unable to lock file, errno = 11, error message = 'Resource temporarily unavailable'

End of HDF5 error back trace

Unable to open/create file 'results.h5'
file '/media/cephfs2/jparham/Joe for Jerome /Divisions with G2toConstriction/01a_DSM639_SyS

In [ ]:
len(res[0])

In [ ]:
df = pd.concat(res[0])

In [ ]:
df.to_csv(dst/'results.csv')